In [9]:
import os
import json
from pathlib import Path
from langchain.text_splitter import TokenTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, Qdrant
from langchain.document_loaders import TextLoader

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [10]:
keys_file = Path(".").resolve().parent / "keys.json"
print(keys_file)
with open(keys_file) as f:
    keys = json.load(f)
os.environ["OPENAI_API_KEY"] = keys["OPENAI_API_KEY"]

D:\00Repos\GPT-Swarm\keys.json


In [11]:
embeddings = OpenAIEmbeddings()

In [12]:
persist_directory = Path("D:\\00Repos\\GPT-Swarm\\tmp\\swarm\\shared_memory")
vectordb = Chroma(persist_directory=str(persist_directory), embedding_function=embeddings)
retriever_chroma = vectordb.as_retriever(search_type="mmr")

Using embedded DuckDB with persistence: data will be stored in: D:\00Repos\GPT-Swarm\tmp\swarm\shared_memory


In [13]:
vectordb._collection.count()

1692

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chains.question_answering import load_qa_chain

In [15]:
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.2) # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa_chain = load_qa_chain(model, chain_type="stuff")
qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=retriever_chroma)

In [ ]:
context = (
    "A new startup just send us their pitch. Find if the startup is worth investing in. The startup is called Brainamics and it is in the space of brain computer interfaces."
)

questions = [
    "Generate a comprehensive description of the startup. Describe their value proposition, the product, USP and business model of a startup.",
    "Find top 10 companies and startups in this field. Find out their locations, raised funding, value proposition, differentiation, etc.",
    "Find top 5 investors in this field. Includ specific details in the format of 'company AAA (link) invested in company BBB (link) $XX in year YYYY'",
    "Describe the market size, growth rate and trends of this field.",
    "Main problems and challenges of the field. Create an extensive list of problems. What can stop the field from growing? What can stop the company from succeeding?",
    "Briefly describe the technology for the non-tech audience. Include links to the main articles in the field.",
    "What questions should we ask the startup to make a more informed decision? Avoid generic and obvious questions and focus on field/domain specific questions that can uncover problems with this specific startup."
]

for question in questions:  
    result = qa.run(context+question+"Be very brief and concise. Focus on the essential information and provide https links to the sources.")
    print(f"=====================================================================================================")
    print(f"=====================================================================================================")
    print(f"{bcolors.OKBLUE }**Question**:{bcolors.ENDC } {question} \n")
    print(f"{bcolors.OKGREEN  }**Answer**{bcolors.ENDC  }: {result} \n")